[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BouleJaune/picselliaT/blob/master/notebookOBDTC.ipynb)



# Object-detection made easy
We will learn how to easily train an object detection model from a list of pre-trained models with the dataset you created on the picsell-IA platform.

In [1]:
!git clone https://github.com/BouleJaune/picselliaT
%cd picselliaT/
!pip install picsellia
%tensorflow_version 1.x

Cloning into 'picselliaT'...
remote: Enumerating objects: 1096, done.
remote: Counting objects: 100% (1096/1096), done.
remote: Compressing objects: 100% (733/733), done.
remote: Total 1096 (delta 377), reused 1068 (delta 354), pack-reused 0
Receiving objects: 100% (1096/1096), 26.03 MiB | 36.27 MiB/s, done.
Resolving deltas: 100% (377/377), done.
/content/picselliaT
  Created wheel for picsellia: filename=picsellia-0.2.41-cp36-none-any.whl size=11393 sha256=aaee0643933229221be7d95b4c2edc12e7acb044b59db2943bdf0537a5530981
  Stored in directory: /root/.cache/pip/wheels/49/cc/ca/5b31617387256247be3e2b9c0f2337ac04ff4ffa5cca06f9b7
Successfully built picsellia
TensorFlow 1.x selected.


In [2]:
import gdown
gdown.download("https://drive.google.com/u/0/uc?export=download&confirm=sWbX&id=14UDmfWrnP2jwppxvcL6wkjPtObuuMh20", "models.zip", quiet=False)
!unzip models.zip

Downloading...
From: https://drive.google.com/u/0/uc?export=download&confirm=sWbX&id=14UDmfWrnP2jwppxvcL6wkjPtObuuMh20
To: /content/picselliaT/models.zip
1.05GB [00:04, 223MB/s]


Archive:  models.zip
   creating: models/
   creating: models/faster_rcnn/
  inflating: models/faster_rcnn/model.ckpt.data-00000-of-00001  
  inflating: models/faster_rcnn/model.ckpt.meta  
  inflating: models/faster_rcnn/checkpoint  
  inflating: models/faster_rcnn/pipeline.config  
  inflating: models/faster_rcnn/frozen_inference_graph.pb  
   creating: models/faster_rcnn/saved_model/
   creating: models/faster_rcnn/saved_model/variables/
  inflating: models/faster_rcnn/saved_model/saved_model.pb  
  inflating: models/faster_rcnn/model.ckpt.index  
   creating: models/mask_rcnn/
  inflating: models/mask_rcnn/model.ckpt.data-00000-of-00001  
  inflating: models/mask_rcnn/model.ckpt.meta  
  inflating: models/mask_rcnn/checkpoint  
  inflating: models/mask_rcnn/pipeline.config  
  inflating: models/mask_rcnn/frozen_inference_graph.pb  
   creating: models/mask_rcnn/saved_model/
   creating: models/mask_rcnn/saved_model/variables/
  inflating: models/mask_rcnn/saved_model/saved_model.pb

## Imports

In [4]:
import sys
sys.path.append("slim")
from picsellia import Client
import main
from util.infer import infer
import tensorflow as tf

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.







## List of possible models : 

- Mask RCNN, a segmentation model. String variable: 'mask_rcnn'
- Faster RCNN an accurate but slow object detection model. String variable: 'faster_rcnn'
- SSD Inception, a fast but less accurate object detection model. String variable : 'ssd_inception'

## Setup

We need to start setting up some variables before anything else.

In [0]:
token = "f2a5daec-e0cc-4ea8-a5eb-10d04fd1e153" # Token from the picsell-IA platform
model_picked = "faster_rcnn" # Choose your base model here from the list of possible models
model_name = "faster_rcnn" # Name your to-be trained model
annotation_type = "rectangle" # Chose the type of annotation used

batch_size = 1
learning_rate = None #You can let this value to None
nb_steps = 20000
mask_type = None #Set this to 'PNG_MASKS' if you want to train a mask segmentation model.

## Client initialisation and data pre-processing

We communicate with the platform to create a new model and get the images and annotations.
With this we can generate the label map, smartly split our data then create the TFRecord files which will be used as input for the model. 

In [6]:
path_models = "models/"
model_selected = path_models + model_picked + "/"

clt = Client(token=token, host="https://backstage.picsellia.com/sdk/")
clt.init_model(model_name)

clt.dl_annotations()
clt.generate_labelmap()
clt.local_pic_save()

main.create_record_files(label_path=clt.label_path, record_dir=clt.record_dir, 
                         tfExample_generator=clt.tf_vars_generator, annotation_type=annotation_type)

Initializing Picsell.ia Client at https://backstage.picsellia.com/sdk/ ...
Connection established at https://backstage.picsellia.com/sdk/
Connection Established
It's your first training for this project
First time using Picsell.ia on this project, initializing directories ...
Creating directory for project 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/
Creating directory for PNG Images of project 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/
Creating directory for checkpoints project 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/
Creating directory for logs of project 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/logs/
Creating directory for records of project 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/
Creating directory for config of project 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/
Creating direct

Here we edit the base protobuf configuration of our model with our parameters. 
We check if it the first training on this model to see if we should train from a previous checkpoint or not.

In [7]:
if clt.training_id!=0:
    previous_path = clt.base_dir.split("/")[:-1]
    previous_path[-1] = clt.training_id - 1
    model_selected = "{}/{}/{}/".format(*previous_path)+"checkpoint/"
    
main.edit_config(model_selected=model_selected, config_output_dir=clt.config_dir,
            record_dir=clt.record_dir, 
            label_map_path=clt.label_path, 
            masks=mask_type, 
            num_steps=nb_steps,
            batch_size=batch_size, 
            learning_rate=learning_rate,
            training_id=clt.training_id)



INFO:tensorflow:Writing pipeline config file to 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/config/pipeline.config


## Training

We can launch the training... and it's as easy as just telling the fonction where is the configuration file and where we want the checkpoints and records to be saved !

In [0]:
main.legacy_train(ckpt_dir=clt.checkpoint_dir, 
                     conf_dir=clt.config_dir)





Instructions for updating:
Please switch to tf.train.create_global_step




Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.Dataset.map()
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "










Instructions for updating:
Please switch to tf.train.get_or_create_global_step

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from models/faster_rcnn/model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path 5e0cbed2-3593-4bf1-8256-b3077725f626/4397ecc5-8dfd-4fe9-b04c-09535d9d4ad4/0/checkpoint/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global

Now that the training has ended we want to send the logs to our dashboard so we can neatly see the sweet decrease of the loss. 

In [0]:
dict_log = main.tfevents_to_dict(path=clt.checkpoint_dir)
clt.send_logs(dict_log)

## Exporting and infering

The model is trained but we still need to export it to a Tensorflow graph proto to use it.

In [0]:
tf.reset_default_graph()
main.export_infer_graph(ckpt_dir=clt.checkpoint_dir, 
                       exported_model_dir=clt.exported_model, 
                       pipeline_config_path=clt.config_dir,
                       write_inference_graph=True, input_type="image_tensor", input_shape=None)

Now we will use the exported model to do some inference on our evaluation set, then send the results to the dashboard.
You can set the minimum confidence treshold at which we keep the bounding boxes to the value you like the most.

In [0]:
min_score_thresh = 0.6 
infer(clt.eval_list, exported_model_dir=clt.exported_model, 
          label_map_path=clt.label_path, results_dir=clt.results_dir, min_score_thresh=min_score_thresh)
clt.send_examples()